<a href="https://colab.research.google.com/github/dsborges2503/Python_Financas_Investimento_Colab/blob/main/SimuladorDeEstrategiaBovespa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install Backtesting

In [ ]:
# Configurando Yahoo Finance
!pip install yfinance --upgrade --no-cache-dir
import yfinance as yf

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import random

In [ ]:
ticker = "WEGE3.SA"

dados_yahoo = yf.download(tickers=ticker, period='1y')
dados_yahoo

In [ ]:
dados_yahoo_index = dados_yahoo.index.tolist()
dados_yahoo['Close'].values.tolist()

In [ ]:
import pandas as pd

acoes = {'Open': dados_yahoo['Open'].values.tolist(),
        'High': dados_yahoo['High'].values.tolist(),
        'Low': dados_yahoo['Low'].values.tolist(),
        'Close': dados_yahoo['Close'].values.tolist(),
        'Volume': dados_yahoo['Volume'].values.tolist()
        }

df = pd.DataFrame(acoes, columns = ['Open', 'High', 'Low', 'Close', 'Volume'], index=dados_yahoo_index)

df

In [ ]:
import pandas as pd


def SMA(values, n):
    return pd.Series(values).rolling(n).mean()

    

In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the two MA lags as *class variables*
    # for later optimization
    n1 = 8
    n2 = 34
    
    def init(self):
        # Precompute the two moving averages
        self.sma1 = self.I(SMA, self.data.Close, self.n1)
        self.sma2 = self.I(SMA, self.data.Close, self.n2)
    
    def next(self):
        # If sma1 crosses above sma2, close any existing
        # short trades, and buy the asset
        if crossover(self.sma1, self.sma2):
            self.position.close()
            self.buy()

        # Else, if sma1 crosses below sma2, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma2, self.sma1):
            self.position.close()
            self.sell()

In [ ]:
%%script echo

    def next(self):
        if (self.sma1[-2] < self.sma2[-2] and
                self.sma1[-1] > self.sma2[-1]):
            self.position.close()
            self.buy()

        elif (self.sma1[-2] > self.sma2[-2] and    # Ugh!
              self.sma1[-1] < self.sma2[-1]):
            self.position.close()
            self.sell()

In [ ]:
from backtesting import Backtest
df=df.dropna()
bt = Backtest(df, SmaCross, cash=10000, commission=0.002)
stats = bt.run()
stats

In [ ]:
bt.plot()

In [ ]:
%%time

stats = bt.optimize(n1=range(5, 150, 5),
                    n2=range(5, 200, 5),
                    maximize='Equity Final [$]',
                    constraint=lambda param: param.n1 < param.n2)
stats

In [ ]:
stats

In [ ]:
stats._strategy

In [ ]:
bt.plot(plot_volume=False, plot_pl=False)

In [4]:
stats.tail()

SyntaxError: ignored

In [ ]:
stats['_equity_curve']

In [ ]:
stats['_trades'] 